In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langsmith import traceable

# 환경 변수 로드
load_dotenv()

# Gemini API 키 설정
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [2]:
# Gemini 모델 초기화
model = genai.GenerativeModel('models/gemini-2.0-flash')

# 트레이싱이 가능한 wrapper 함수 생성
@traceable
def generate_with_gemini(prompt: str):
    response = model.generate_content(prompt)
    return response.text

# 테스트 실행
result = generate_with_gemini('계란찜 만드는 방법을 알려줘')
print(result)

네, 부드럽고 맛있는 계란찜 만드는 방법 알려드릴게요! 다양한 방법이 있지만, 가장 기본적인 방법과 전자레인지로 간단하게 만드는 방법, 그리고 좀 더 특별한 맛을 내는 방법까지 알려드릴게요.

**1. 기본 계란찜 (뚝배기 또는 냄비)**

**재료:**

*   계란 3개
*   물 또는 다시마 육수 1/2컵 (계란 양의 1/2 ~ 2/3)
*   소금 1/4 작은술 (또는 국간장 1/2 작은술)
*   (선택) 다진 파, 다진 당근, 참기름, 깨소금 약간

**만드는 법:**

1.  **계란물 만들기:** 계란을 깨서 볼에 넣고 소금을 넣은 후 거품기로 잘 풀어줍니다. 알끈을 제거하기 위해 체에 한 번 걸러주면 더욱 부드러워요.
2.  **물(육수) 섞기:** 계란물에 물 또는 다시마 육수를 넣고 잘 섞어줍니다.
3.  **간 맞추기:** 소금으로 간을 맞추고, (선택 사항) 다진 파, 다진 당근 등 원하는 재료를 넣어 섞습니다.
4.  **찜 냄비에 끓이기:** 뚝배기나 냄비에 계란물을 붓고 약불로 가열합니다.
5.  **뭉치지 않게 저어주기:** 계란찜이 몽글몽글해지기 시작하면 젓가락으로 살살 저어줍니다. 바닥에 눌어붙지 않도록 계속 저어주는 것이 중요해요.
6.  **약불로 익히기:** 겉부분이 익기 시작하면 뚜껑을 덮고 아주 약한 불로 줄여 속까지 익혀줍니다. 뚜껑을 살짝 열어두면 넘치는 것을 방지할 수 있습니다.
7.  **마무리:** 계란찜이 다 익으면 불을 끄고 참기름을 살짝 두르고 깨소금을 뿌려 마무리합니다.

**팁:**

*   계란찜이 너무 빨리 익으면 윗부분이 갈라질 수 있으니 불 조절이 중요합니다.
*   젓가락으로 찔러보아 묻어 나오는 것이 없으면 다 익은 것입니다.

**2. 전자레인지 계란찜**

**재료:**

*   계란 2개
*   물 1/4컵 (계란 양의 1/2)
*   소금 약간
*   (선택) 다진 파, 참기름, 깨소금 약간

**만드는 법:**

1.  **계란물 만들기:** 전자레인지용 그릇에 계란을 깨서 넣고 

In [3]:
from IPython.display import Markdown

Markdown(result)

네, 부드럽고 맛있는 계란찜 만드는 방법 알려드릴게요! 다양한 방법이 있지만, 가장 기본적인 방법과 전자레인지로 간단하게 만드는 방법, 그리고 좀 더 특별한 맛을 내는 방법까지 알려드릴게요.

**1. 기본 계란찜 (뚝배기 또는 냄비)**

**재료:**

*   계란 3개
*   물 또는 다시마 육수 1/2컵 (계란 양의 1/2 ~ 2/3)
*   소금 1/4 작은술 (또는 국간장 1/2 작은술)
*   (선택) 다진 파, 다진 당근, 참기름, 깨소금 약간

**만드는 법:**

1.  **계란물 만들기:** 계란을 깨서 볼에 넣고 소금을 넣은 후 거품기로 잘 풀어줍니다. 알끈을 제거하기 위해 체에 한 번 걸러주면 더욱 부드러워요.
2.  **물(육수) 섞기:** 계란물에 물 또는 다시마 육수를 넣고 잘 섞어줍니다.
3.  **간 맞추기:** 소금으로 간을 맞추고, (선택 사항) 다진 파, 다진 당근 등 원하는 재료를 넣어 섞습니다.
4.  **찜 냄비에 끓이기:** 뚝배기나 냄비에 계란물을 붓고 약불로 가열합니다.
5.  **뭉치지 않게 저어주기:** 계란찜이 몽글몽글해지기 시작하면 젓가락으로 살살 저어줍니다. 바닥에 눌어붙지 않도록 계속 저어주는 것이 중요해요.
6.  **약불로 익히기:** 겉부분이 익기 시작하면 뚜껑을 덮고 아주 약한 불로 줄여 속까지 익혀줍니다. 뚜껑을 살짝 열어두면 넘치는 것을 방지할 수 있습니다.
7.  **마무리:** 계란찜이 다 익으면 불을 끄고 참기름을 살짝 두르고 깨소금을 뿌려 마무리합니다.

**팁:**

*   계란찜이 너무 빨리 익으면 윗부분이 갈라질 수 있으니 불 조절이 중요합니다.
*   젓가락으로 찔러보아 묻어 나오는 것이 없으면 다 익은 것입니다.

**2. 전자레인지 계란찜**

**재료:**

*   계란 2개
*   물 1/4컵 (계란 양의 1/2)
*   소금 약간
*   (선택) 다진 파, 참기름, 깨소금 약간

**만드는 법:**

1.  **계란물 만들기:** 전자레인지용 그릇에 계란을 깨서 넣고 소금을 넣은 후 잘 풀어줍니다.
2.  **물 섞기:** 계란물에 물을 넣고 잘 섞어줍니다.
3.  **간 맞추기:** 소금으로 간을 맞추고, (선택 사항) 다진 파 등 원하는 재료를 넣어 섞습니다.
4.  **전자레인지에 돌리기:** 랩을 씌우거나 뚜껑을 덮고 (숨구멍을 몇 개 뚫어주세요) 전자레인지에 2~3분 돌립니다. (전자레인지 사양에 따라 시간이 달라질 수 있습니다.)
5.  **상태 확인 및 추가 가열:** 꺼내서 상태를 확인하고, 덜 익었으면 30초씩 추가로 돌려줍니다.
6.  **마무리:** 참기름을 살짝 두르고 깨소금을 뿌려 마무리합니다.

**팁:**

*   전자레인지용 그릇은 유리나 도자기 재질을 사용하세요.
*   계란찜이 부풀어 오르면서 넘칠 수 있으니, 깊이가 있는 그릇을 사용하는 것이 좋습니다.

**3. 특별한 계란찜 (새우젓 계란찜)**

**재료:**

*   계란 3개
*   물 또는 다시마 육수 1/2컵
*   새우젓 1 작은술 (다져서 사용)
*   다진 마늘 약간
*   (선택) 청양고추 약간 (송송 썰기), 참기름, 깨소금

**만드는 법:**

1.  기본 계란찜과 동일하게 계란물을 만들고, 물(육수)을 섞어줍니다.
2.  새우젓을 다져서 넣고, 다진 마늘을 넣어 섞어줍니다.
3.  (선택 사항) 청양고추를 넣어 매콤한 맛을 더할 수 있습니다.
4.  기본 계란찜과 동일하게 뚝배기나 냄비에 끓여줍니다.
5.  마지막에 참기름과 깨소금을 뿌려 마무리합니다.

**팁:**

*   새우젓은 짠맛이 강하므로, 소금 양을 조절해야 합니다.
*   새우젓 대신 명란젓을 사용해도 맛있습니다.

**주의사항:**

*   계란 알레르기가 있는 경우 섭취를 피해주세요.
*   조리 시 화상에 주의하세요.

어떤 방법으로 만들어보고 싶으신가요? 궁금한 점이 있다면 언제든지 물어보세요! 😊


In [4]:
from langchain_community.document_loaders import PyMuPDFLoader
# 환경 변수 로드
load_dotenv()

# @traceable
def load_pdf(pdf_path: str):
    """
    PDF 파일을 로드하는 함수
    
    Args:
        pdf_path (str): PDF 파일의 경로
        
    Returns:
        list: Document 객체 리스트
    """
    try:
        # PDF 로더 초기화
        loader = PyMuPDFLoader(pdf_path)
        
        # PDF 로드
        documents = loader.load()
        
        print(f"PDF 로드 완료: {len(documents)} 페이지")
        return documents
    
    except Exception as e:
        print(f"PDF 로드 중 오류 발생: {str(e)}")
        return None

In [5]:
from glob import glob

current_dir = os.getcwd()

pdf_paths = glob(os.path.join(current_dir, "document", "*.pdf"))

In [6]:
import json

docs = []
for pdf_path in pdf_paths:
    title = pdf_path.split('\\')[-1]
    print(f"PDF 로드 시작: {title}")
    documents = load_pdf(pdf_path)
    for document in documents:
        document.metadata['title'] = title
        docs.append(document)

PDF 로드 시작: 20250603_개혁신당_정당정책.pdf
PDF 로드 완료: 11 페이지
PDF 로드 시작: 20250603_국민의힘_정당정책.pdf
PDF 로드 완료: 19 페이지
PDF 로드 시작: 20250603_더불어민주당_정당정책.pdf
PDF 로드 완료: 20 페이지


In [7]:
print(len(docs))
docs

50


[Document(metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13015', 'creationdate': '2025-05-10T14:48:01+09:00', 'source': 'c:\\Users\\LeeSeungYong\\myproject\\ai-agent\\agentic-rag\\document\\20250603_개혁신당_정당정책.pdf', 'file_path': 'c:\\Users\\LeeSeungYong\\myproject\\ai-agent\\agentic-rag\\document\\20250603_개혁신당_정당정책.pdf', 'total_pages': 11, 'format': 'PDF 1.4', 'title': '20250603_개혁신당_정당정책.pdf', 'author': 'USER', 'subject': '', 'keywords': '', 'moddate': '2025-05-10T14:48:01+09:00', 'trapped': '', 'modDate': "D:20250510144801+09'00'", 'creationDate': "D:20250510144801+09'00'", 'page': 0}, page_content="- 1 -\n선거명\n제21대 대통령선거\n정당명\n개혁신당\n공약순위 : 1\n제   목 : 대통령 힘빼고 일 잘하는 정\n부 만든다\n*분    야행정\n○ 목 표\n ◦ 부처 간 소관 분야 중복과 행정의 칸막이 문제를 최소화하여 효율적이고 \n전문화된 정부 운영\n ◦ 실무 중심의 작은 정부 기조 확립 및 부처 이름이 아닌 실제 업무 성과와 \n전문성을 중시하는 문화 정착\n○ 이행방법\n ① 부처 개편 및 축소 (19부처 → 13부처)\n  ◦ 유사·중복 업무 부처 통폐합\n   · 교육부와 과학기술정보통신부 통합 (교육과학부)\n   · 통일부 폐지 및 외교부로 업무 통합 (외교통일부)\n   · 여성가족부 폐지 및 관련 업무 복지부와

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=100, model_name = "gpt-4"
)
doc_splits = text_splitter.split_documents(docs)

In [9]:
import tiktoken

print(len(doc_splits))
# doc_splits[4].page_content.strip()
for i, split in enumerate(doc_splits):
    encoder = tiktoken.get_encoding("cl100k_base")
    tokens = encoder.encode(split.page_content)
    print(f"{i+1}번째 Document 토큰 개수: {len(tokens)}")
    print(split.page_content)
    print()

57
1번째 Document 토큰 개수: 926
- 1 -
선거명
제21대 대통령선거
정당명
개혁신당
공약순위 : 1
제   목 : 대통령 힘빼고 일 잘하는 정
부 만든다
*분    야행정
○ 목 표
 ◦ 부처 간 소관 분야 중복과 행정의 칸막이 문제를 최소화하여 효율적이고 
전문화된 정부 운영
 ◦ 실무 중심의 작은 정부 기조 확립 및 부처 이름이 아닌 실제 업무 성과와 
전문성을 중시하는 문화 정착
○ 이행방법
 ① 부처 개편 및 축소 (19부처 → 13부처)
  ◦ 유사·중복 업무 부처 통폐합
   · 교육부와 과학기술정보통신부 통합 (교육과학부)
   · 통일부 폐지 및 외교부로 업무 통합 (외교통일부)
   · 여성가족부 폐지 및 관련 업무 복지부와 내무부(행정안전부)로 이관
   · 보건부를 별도 분리 신설하여 보건의료 분야 전문성 강화
   · 국가보훈부를 복지부로 개편하여 복지정책과 통합 운영
   · 산업자원통상부와 중소기업벤처부 통합하여 산업에너지부로 일원화
   · 국토교통부, 환경부, 해양수산부의 해양 업무를 건설교통부로 통합하고, 
수산 분야는 일차산업부로 통합
   · 문화체육관광부, 기획재정부, 행정안전부를 각각 문화부, 재정경제부, 내
무부로 명칭 변경
 ② 3부총리제 도입
  ◦ 안보부총리, 전략부총리, 사회부총리를 각각 임명하여 책임운영체제 구축
  ◦ 대통령 산하 국가안보실 폐지, 안보부총리가 해당 기능 수행
 ③ 예산 편성 기능 이관
  ◦ 기획재정부에서 예산기획 기능을 분리하여 국무총리실 산하 '예산기획실' 신설
  ◦ 각 부처 예산 총액 배정 국무회의에서 의결하여 특정 부처의 예산 독점 방지
 ④ 정부기구 효율화
  ◦ 고위공직자범죄수사처(공수처) 폐지
  ◦ 국가인권위원회와 국민권익위원회 통합하여 기능적 효율성 극대화

2번째 Document 토큰 개수: 285
- 2 -
○ 이행기간
 ◦ 부처 개편 계획 수립 및 관련 법령 개정안 마련: 취임 후 6개월 내
 ◦ 국회 법령 개정 및 예산 심의 통과: 취임 후 1년 이내

In [10]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Gemini Embedding 모델 초기화
embeddings = GoogleGenerativeAIEmbeddings(
    # model="models/gemini-embedding-exp-03-07",  # Gemini의 embedding 모델
    model="models/text-embedding-004"
)

# 벡터 스토어 생성
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=embeddings
)

In [11]:
# 검색기 생성
retriever = vectorstore.as_retriever()

# 테스트 검색 (새로운 방식)
query = "더불어 민주당의 친환경 에너지 정책이 뭔지 설명해줘"
# relevant_documents = retriever.invoke(query)  # get_relevant_documents 대신 invoke 사용

In [12]:
from IPython.display import display, Markdown

def print_documents(documents):
    """
    검색된 문서들을 보기 좋게 출력하는 함수
    """
    for i, doc in enumerate(documents, 1):
        # 마크다운 형식으로 출력
        display(Markdown(f"""
### 문서 {i}
---
{doc.page_content}
---
        """))
        
        # 메타데이터가 있다면 출력
        if doc.metadata:
            print("\n메타데이터:")
            for key, value in doc.metadata.items():
                if key == "title":
                    print(f"- {key}: {value}")
        print("\n" + "="*80 + "\n")  # 구분선

In [13]:
# 사용 예시
# print_documents(relevant_documents)

In [34]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "policy_search",
    "각 정당 별 정책을 검색하고 정보를 출력합니다.",
)

In [35]:
retriever_tool.invoke({
    "query" : "국민의 힘의 경제 정책에 대해서 설명해줘."
})

'○건축물·열 부문 탈탄소화\n- 민간·공공 그린리모델링 지원 확대 및 절차 간소화를 통한 노후건물 에너지효율화\n○전기차 보급 확대 및 노후경유차 조기 대·폐차 지원을 통한 수송부문 탈탄소 가속화\n\n- 항공사고 예방을 위한 항공사·공항시설 안전관련 투자·정비 점검 강화 \n- 건설공사 발주ㆍ설계ㆍ시공ㆍ감리 등 전 과정에 대한 안전대책 강화\n○지역·필수·공공의료 강화로 제대로 치료받을 권리 확보\n\n- 저연차 공무원의 보수 지속적 인상, 경찰‧소방‧재난담당 공무원 위험근무수당 인상\n- ‘간부 모시는 날’, 불합리한 업무 지시 등 잘못된 공직관행 혁신\n○문화예술인 창작권 보장을 위한 권리 강화 및 정부의 문화예술인 창작권 침해 금지\n\n- 지역과 함께 성장하고 국립대-사립대가 동반성장하는 RISE 체계 구축 \n○지역사랑상품권 발행 의무화를 통해 지역경제를 살리고 균형발전 달성\n○‘잘사니즘’의 실현, 관광산업으로 지역경제 활성화'

In [36]:
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model

response_model = init_chat_model(
    "google_genai:gemini-2.0-flash",
    temperature=0
)

def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    response = (
        response_model.bind_tools([retriever_tool]).invoke(state["messages"])
    )
    return {"messages": [response]}

In [37]:
input = {"messages": [{"role": "user", "content": "hello!"}]}
response = generate_query_or_respond(input)["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello! How can I help you today?


In [40]:
input = {
    "messages": [
        {
            "role": "user",
            "content": "개혁신당의 경제 정책에 뭐가 있는지 찾아줘",
        }
    ]
}
response = generate_query_or_respond(input)

In [43]:
response["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  policy_search (e9fc9b95-7afa-43cc-8b74-4316497e82a8)
 Call ID: e9fc9b95-7afa-43cc-8b74-4316497e82a8
  Args:
    query: 개혁신당 경제 정책


In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

GRADE_PROMPT = (
    "You are a grader assessing relevance of a retrieved document to a user question. \n "
    "Here is the retrieved document: \n\n {context} \n\n"
    "Here is the user question: {question} \n"
    "If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n"
    "Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."
)


class GradeDocuments(BaseModel):
    """Grade documents using a binary score for relevance check."""

    binary_score: str = Field(
        description="Relevance score: 'yes' if relevant, or 'no' if not relevant"
    )


grader_model = init_chat_model("openai:gpt-4.1", temperature=0)


def grade_documents(
    state: MessagesState,
) -> Literal["generate_answer", "rewrite_question"]:
    """Determine whether the retrieved documents are relevant to the question."""
    question = state["messages"][0].content
    context = state["messages"][-1].content

    prompt = GRADE_PROMPT.format(question=question, context=context)
    response = (
        grader_model
        .with_structured_output(GradeDocuments).invoke(
            [{"role": "user", "content": prompt}]
        )
    )
    score = response.binary_score

    if score == "yes":
        return "generate_answer"
    else:
        return "rewrite_question"

In [ ]:
from langchain_core.messages import convert_to_messages

input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "What does Lilian Weng say about types of reward hacking?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "types of reward hacking"},
                    }
                ],
            },
            {"role": "tool", "content": "meow", "tool_call_id": "1"},
        ]
    )
}
grade_documents(input)

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "What does Lilian Weng say about types of reward hacking?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "types of reward hacking"},
                    }
                ],
            },
            {
                "role": "tool",
                "content": "reward hacking can be categorized into two types: environment or goal misspecification, and reward tampering",
                "tool_call_id": "1",
            },
        ]
    )
}
grade_documents(input)